In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../data/2022_public_lar.csv")

/Users/jihan/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080210 entries, 0 to 16080209
Data columns (total 99 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   activity_year                             int64  
 1   lei                                       object 
 2   derived_msa_md                            int64  
 3   state_code                                object 
 4   county_code                               float64
 5   census_tract                              float64
 6   conforming_loan_limit                     object 
 7   derived_loan_product_type                 object 
 8   derived_dwelling_category                 object 
 9   derived_ethnicity                         object 
 10  derived_race                              object 
 11  derived_sex                               object 
 12  action_taken                              int64  
 13  purchaser_type                            int64  
 14  

In [4]:
def drop_cols(df, cols):
    return df.drop(columns=cols)

def impute_numeric_cols(df, cols, strategy='median'):
    imputer = SimpleImputer(strategy=strategy)
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = imputer.fit_transform(df[[col]]).ravel()
    return df

def recode_categorical_cols(df, recode_map):
    for col, mapping in recode_map.items():
        df[col] = df[col].replace(mapping)
    return df

In [5]:
# Drop unnecessary, redundant, and sparse columns (based on domain knowledge)
unnecessary = df.columns[list(range(47, 77)) + list(range(78, 99)) + [0, 1, 13, 18]]
redundant = ['census_tract', 'state_code', 'county_code', 'loan_type', 'lien_status', 'construction_method']
sparse = ['discount_points', 'total_points_and_fees', 'lender_credits', 'prepayment_penalty_term', 'intro_rate_period', 'multifamily_affordable_units']
df = drop_cols(df, unnecessary)
df = drop_cols(df, redundant)
df = drop_cols(df, sparse)

In [7]:
 # Drop NAs
df = df.dropna()

In [9]:
df = df[(df['derived_ethnicity'] != 'Ethnicity Not Available') & (df['derived_ethnicity'] != 'Free Form Text Only')]
df = df[(df['derived_race'] != 'Race Not Available') & (df['derived_race'] != 'Free Form Text Only')]
df = df[df['derived_sex'] != 'Sex Not Available']
df = df[df['applicant_age'] != '8888']

In [11]:
df = df[df['action_taken'].isin([1, 2, 3, 7, 8])]
df['loan_approved'] = df['action_taken'].apply(lambda x: 1 if x in [1, 2, 8] else 0)
df = drop_cols(df, ['action_taken'])

In [13]:
numeric_cols = ['total_loan_costs', 'origination_charges', 'loan_term', 'property_value', 'combined_loan_to_value_ratio', 'interest_rate', 'rate_spread']
df = impute_numeric_cols(df, numeric_cols)

In [14]:
rename_map = {'derived_msa_md': 'msa_md', 'derived_loan_product_type': 'loan_product_type', 'derived_dwelling_category': 'dwelling_category', 'derived_ethnicity': 'ethnicity', 'derived_race': 'race', 'derived_sex': 'sex', 'applicant_age': 'age'}
df = df.rename(columns=rename_map).astype({'msa_md': object, 'loan_term': int, 'total_units': int, 'loan_approved': object})

In [16]:
recode_map = {
    'ethnicity': {'Not Hispanic or Latino': 'Non-Hispanic', 'Hispanic or Latino': 'Hispanic'},
    'race': {'Black or African American': 'Black', 'American Indian or Alaska Native': 'Native',
                'Native Hawaiian or Other Pacific Islander': 'PI', '2 or more minority races': '2+Minority'},
    'preapproval': {2: 0},
    'loan_purpose': {1: 'Purchase', 2: 'Improvement', 31: 'Refinancing', 32: 'Cash-out', 4: 'Other', 5: 'N/a'},
    'open_end_line_of_credit': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'business_or_commercial_purpose': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'hoepa_status': {1: 'Yes', 2: 'No', 3: 'N/a'},
    'negative_amortization': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'interest_only_payment': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'balloon_payment': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'other_nonamortizing_features': {1: 'Yes', 2: 'No', 1111: 'Exempt'},
    'occupancy_type': {1: 'Principal', 2: 'Second', 3: 'Investment'},
    'manufactured_home_secured_property_type': {1: 'Home&Land', 2: 'Home', 3: 'N/a', 1111: 'Exempt'},
    'manufactured_home_land_property_interest': {1: 'Direct', 2: 'Indirect', 3: 'Paid', 4: 'Unpaid', 5: 'N/a', 1111: 'Exempt'},
    'debt_to_income_ratio': {39.0: '39', 38.0: '38', 37.0: '37'}
    }
df = recode_categorical_cols(df, recode_map)
df['preapproval'] = df['preapproval'].astype(object)

In [18]:
scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['number'])
df[numeric_cols.columns] = scaler.fit_transform(numeric_cols)

In [21]:
categorical_cols = list(range(0, 7)) + list(range(8, 11)) + [15] + list(range(19, 23)) + list(range(24, 27)) + list(range(29, 31))
encoded = pd.get_dummies(df.iloc[:, categorical_cols])
df = df.reset_index(drop=True)
encoded = encoded.reset_index(drop=True)
df = pd.concat([df, encoded], axis=1)
df = df.drop(df.columns[categorical_cols], axis=1)

In [23]:
df.to_csv("../data/preprocessed_data.csv", index=False)

In [3]:
df = pd.read_csv("../data/preprocessed_data.csv")

In [4]:
sampled_df = df.sample(50000, random_state=334)

In [5]:
sampled_df = sampled_df.reset_index(drop=True)

In [7]:
sampled_df.to_csv("../data/sampled_preprocessed_data.csv", index=False)

In [46]:
total = df['race'].value_counts(normalize=True)
approved = df[df['loan_approved'] == 1]['race'].value_counts(normalize=True)
denied = df[df['loan_approved'] == 0]['race'].value_counts(normalize=True)

total /= total.sum()
approved /= approved.sum()
denied /= denied.sum()

proportions_race = pd.DataFrame({
    'Total': total,
    'Approved': approved,
    'Denied': denied
})

In [47]:
proportions_race

,Total,Approved,Denied
2+Minority,0.002028,0.002028,0.002059
Asian,0.070292,0.070468,0.033847
Black,0.087483,0.087427,0.099095
Joint,0.027461,0.027510,0.017245
Native,0.005905,0.005869,0.013298
PI,0.002153,0.002153,0.002016
White,0.804677,0.804544,0.832440


In [50]:
total = df['ethnicity'].value_counts(normalize=True)
approved = df[df['loan_approved'] == 1]['ethnicity'].value_counts(normalize=True)
denied = df[df['loan_approved'] == 0]['ethnicity'].value_counts(normalize=True)

total /= total.sum()
approved /= approved.sum()
denied /= denied.sum()

proportions_ethnicity = pd.DataFrame({
    'Total': total,
    'Approved': approved,
    'Denied': denied
})

In [51]:
proportions_ethnicity

,Total,Approved,Denied
Non-Hispanic,0.866073,0.866010,0.879156
Hispanic,0.104680,0.104697,0.101154
Joint,0.029247,0.029293,0.019690


In [52]:
total = df['sex'].value_counts(normalize=True)
approved = df[df['loan_approved'] == 1]['sex'].value_counts(normalize=True)
denied = df[df['loan_approved'] == 0]['sex'].value_counts(normalize=True)

total /= total.sum()
approved /= approved.sum()
denied /= denied.sum()

proportions_sex = pd.DataFrame({
    'Total': total,
    'Approved': approved,
    'Denied': denied
})

In [53]:
proportions_sex

,Total,Approved,Denied
Female,0.238608,0.238420,0.277723
Joint,0.416273,0.416542,0.360431
Male,0.345118,0.345038,0.361846


In [54]:
total = df['age'].value_counts(normalize=True)
approved = df[df['loan_approved'] == 1]['age'].value_counts(normalize=True)
denied = df[df['loan_approved'] == 0]['age'].value_counts(normalize=True)

total /= total.sum()
approved /= approved.sum()
denied /= denied.sum()

proportions_age = pd.DataFrame({
    'Total': total,
    'Approved': approved,
    'Denied': denied
})

In [55]:
proportions_age

,Total,Approved,Denied
25-34,0.241580,0.241908,0.173266
35-44,0.253356,0.253455,0.232680
45-54,0.201852,0.201720,0.229205
55-64,0.152687,0.152515,0.188495
65-74,0.083767,0.083701,0.097551
<25,0.040050,0.040026,0.044872
>74,0.026709,0.026675,0.033932


## Columns
* `activity_year`: The calendar year the data submission covers
* `lei`: A financial institution’s Legal Entity Identifier
* `derived_msa-md`: The 5 digit derived MSA (metropolitan statistical area) or MD (metropolitan division) code. An MSA/MD is an area that has at least one urbanized area of 50,000 or more population.
* `state_code`: Two-letter state code
* `country_code`: State-county FIPS code
* `census_tract`: 11 digit census tract number
* `derived_loan_product_type`: Derived loan product type from Loan Type and Lien Status fields for easier querying of specific records
    - Conventional:First Lien
    - FHA:First Lien
    - VA:First Lien
    - FSA/RHS:First Lien
    - Conventional:Subordinate Lien
    - FHA:Subordinate Lien
    - VA:Subordinate Lien
    - FSA/RHS:Subordinate Lien
* `derived_dwelling_category`: Derived dwelling type from Construction Method and Total Units fields for easier querying of specific records
    - Single Family (1-4 Units):Site-Built
    - Multifamily:Site-Built (5+ Units)
    - Single Family (1-4 Units):Manufactured
    - Multifamily:Manufactured (5+ Units)
* `conforming_loan_limit`: Indicates whether the reported loan amount exceeds the GSE (government sponsored enterprise) conforming loan limit
    - C (Conforming)
    - NC (Nonconforming)
    - U (Undetermined)
    - NA (Not Applicable)
* `derived_ethnicity`: Single aggregated ethnicity categorization derived from applicant/borrower and co-applicant/co-borrower ethnicity fields
    - Hispanic or Latino
    - Not Hispanic or Latino
    - Joint
    - Ethnicity Not Available
    - Free Form Text Only
* `derived_race`: Single aggregated race categorization derived from applicant/borrower and co-applicant/co-borrower race fields
    - American Indian or Alaska Native
    - Asian
    - Black or African American
    - Native Hawaiian or Other Pacific Islander
    - White
    - 2 or more minority races
    - Joint
    - Free Form Text Only
    - Race Not Available
* `derived_sex`: Single aggregated sex categorization derived from applicant/borrower and co-applicant/co-borrower sex fields
    - Male
    - Female
    - Joint
    - Sex Not Available
* `action_taken`: The action taken on the covered loan or application
    - 1 - Loan originated
    - 2 - Application approved but not accepted
    - 3 - Application denied
    - 4 - Application withdrawn by applicant
    - 5 - File closed for incompleteness
    - 6 - Purchased loan
    - 7 - Preapproval request denied
    - 8 - Preapproval request approved but not accepted
* `purchaser_type`: Type of entity purchasing a covered loan from the institution
    - 0 - Not applicable
    - 1 - Fannie Mae
    - 2 - Ginnie Mae
    - 3 - Freddie Mac
    - 4 - Farmer Mac
    - 5 - Private securitizer
    - 6 - Commercial bank, savings bank, or savings association
    - 71 - Credit union, mortgage company, or finance company
    - 72 - Life insurance company
    - 8 - Affiliate institution
    - 9 - Other type of purchaser
* `preapproval`: Whether the covered loan or application involved a request for a preapproval of a home purchase loan under a preapproval program
    - 1 - Preapproval requested
    - 2 - Preapproval not requested
* `loan_type`: The type of covered loan or application
    - 1 - Conventional (not insured or guaranteed by FHA, VA, RHS, or FSA)
    - 2 - Federal Housing Administration insured (FHA)
    - 3 - Veterans Affairs guaranteed (VA)
    - 4 - USDA Rural Housing Service or Farm Service Agency guaranteed (RHS or FSA)
* `loan_purpose`
    - 1 - Home purchase
    - 2 - Home improvement
    - 31 - Refinancing
    - 32 - Cash-out refinancing
    - 4 - Other purpose
    - 5 - Not applicable
* `lien_status`
    - 1 - Secured by a first lien
    - 2 - Secured by a subordinate lien
* `reverse_mortgage`: Whether the covered loan or application is for a reverse mortgage
    - 1 - Reverse mortgage
    - 2 - Not a reverse mortgage
    - 1111 - Exempt
* `open-end_line_of_credit`: Whether the covered loan or application is for an open-end line of credit
    - 1 - Open-end line of credit
    - 2 - Not an open-end line of credit
    - 1111 - Exempt
* `business_or_commercial_purpose`: Whether the covered loan or application is primarily for a business or commercial purpose
    - 1 - Primarily for a business or commercial purpose
    - 2 - Not primarily for a business or commercial purpose
    - 1111 - Exempt
* `loan_amount`: The amount of the covered loan, or the amount applied for
* `combined_loan_to_value_ratio`: The ratio of the total amount of debt secured by the property to the value of the property relied on in making the credit decision
* `interest_rate`: The interest rate for the covered loan or application
* `rate_spread`: The difference between the covered loan’s annual percentage rate (APR) and the average prime offer rate (APOR) for a comparable transaction as of the date the interest rate is set
* `hoepa_status`: Whether the covered loan is a high-cost mortgage
    - 1 - High-cost mortgage
    - 2 - Not a high-cost mortgage
    - 3 - Not applicable
* `total_loan_costs`: The amount, in dollars, of total loan costs
* `total_points_and_fees`: The total points and fees, in dollars, charged in connection with the covered loan
* `origination_charges`: The total of all itemized amounts, in dollars, that are designated borrower-paid at or before closing
* `discount_points`: The points paid, in dollars, to the creditor to reduce the interest rate
* `lender_credits`: The amount, in dollars, of lender credits
* `loan_term`: The number of months after which the legal obligation will mature or terminate, or would have matured or terminated
* `prepayment_penalty_term`: The term, in months, of any prepayment penalty
* `intro_rate_period`: The number of months, or proposed number of months in the case of an application, until the first date the interest rate may change after closing or account opening
* `negative_amortization`: Whether the contractual terms include, or would have included, a term that would cause the covered loan to be a negative amortization loan
    - 1 - Negative amortization
    - 2 - No negative amortization
    - 1111 - Exempt
* `interest_only_payment`: Whether the contractual terms include, or would have included, interest-only payments
    - 1 - Interest-only payments
    - 2 - No interest-only payments
    - 1111 - Exempt
* `balloon_payment`: Whether the contractual terms include, or would have included, a balloon payment
    - 1 - Balloon payment
    - 2 - No balloon payment
    - 1111 - Exempt
* `other_nonamortizing_features`: Whether the contractual terms include, or would have included, any term, other than those described in Paragraphs 1003.4(a)(27)(i), (ii), and (iii) that would allow for payments other than fully amortizing payments during the loan term
    - 1 - Other non-fully amortizing features
    - 2 - No other non-fully amortizing features
    - 1111 - Exempt
* `property_value`: The value of the property securing the covered loan or, in the case of an application, proposed to secure the covered loan, relied on in making the credit decision
* `construction_method`: Construction method for the dwelling
    - 1 - Site-built
    - 2 - Manufactured home
* `occupancy_type`: Occupancy type for the dwelling
    - 1 - Principal residence
    - 2 - Second residence
    - 3 - Investment property
* `manufactured_home_secured_property_type`: Whether the covered loan or application is, or would have been, secured by a manufactured home and land, or by a manufactured home and not land
    - 1 - Manufactured home and land
    - 2 - Manufactured home and not land
    - 3 - Not applicable
    - 1111 - Exempt
* `manufactured_home_land_property_interest`: The applicant’s or borrower’s land property interest in the land on which a manufactured home is, or will be, located
    - 1 - Direct ownership
    - 2 - Indirect ownership
    - 3 - Paid leasehold
    - 4 - Unpaid leasehold
    - 5 - Not applicable
    - 1111 - Exempt
* `total_units`: The number of individual dwelling units related to the property securing the covered loan or, in the case of an application, proposed to secure the covered loan
    - 1
    - 2
    - 3
    - 4
    - 5-24
    - 25-49
    - 50-99
    - 100-149
    - \>149
* `ageapplicant`: The age of the applicant
    - <25
    - 25-34
    - 35-44
    - 45-54 
    - 55-64
    - 65-74
    - \>74
    - 8888
* `multifamily_affordable_units`: Reported values as a percentage, rounded to the nearest whole number, of the value reported for Total Units 
* `income`: The gross annual income, in thousands of dollars, relied on in making the credit decision, or if a credit decision was not made, the gross annual income relied on in processing the application
* `debt_to_income_ratio`: The ratio, as a percentage, of the applicant’s or borrower’s total monthly debt to the total monthly income relied on in making the credit decision
    - <20%
    - 20%-<30%
    - 30%-<36%
    - 37%
    - 38%
    - 39%
    - 40%
    - 41%
    - 42%
    - 43%
    - 44%
    - 45%
    - 46%
    - 47%
    - 48%
    - 49%
    - 50%-60%
    - \>60%
    - NA
    - Exempt


## Steps
1. Identify Protected Attributes (e.g., race, age, gender, income level, etc.)
2. Data Cleaning (e.g., handling missing values, removing duplicates, and correcting errors)
3. Feature Selection
4. Data Transformation (e.g., standardization, one-hot encoding, etc.)
5. Exploratory Data Analysis (e.g., summary, visualization, bias exploration)
6. Data Splitting
7. Model Selection (e.g., logistic regression, decision trees, random forest, XGBoost, neural networks)
8. Hyperparamter Tuning
9. Model Evaluation
10. Suggestion on Bias Mitigation